In [38]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer

In [39]:
#import raw data
house_data = pd.read_csv('./raw_data/train_data.csv')

In [40]:
# first 5 row of the dataset
house_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


There are missing values in 'Alley', 'PoolQC', Fence, 'MiscFeature' columns in the first 5 rows. 'Id' column is not a predictor in the dataset, but I will keep it for unique identify for data seperation process.

In [41]:
# data types in the data set
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

There are 37 columns containing numeric observations and 38 objects columns, and 1460 objects (houses).

I will use pd.concat to create the dataframe indicating the number of missing values in each column of the dataset and theirs percentage

In [43]:
missing = pd.concat([house_data.isnull().sum(), 100 * house_data.isnull().mean()], axis=1)
missing.columns=['Count', '%']
missing.sort_values(by= 'Count', ascending= False)

,Count,%
PoolQC,1453,99.520548
MiscFeature,1406,96.301370
Alley,1369,93.767123
Fence,1179,80.753425
FireplaceQu,690,47.260274
...,...,...
ExterQual,0,0.000000
Exterior2nd,0,0.000000
Exterior1st,0,0.000000
RoofMatl,0,0.000000


In [73]:
# features that contains about 50% or higher missing values
missing[round(missing['%'], ndigits = -1) >= 50].sort_values(by= 'Count', ascending= False)

,Count,%
PoolQC,1453,99.520548
MiscFeature,1406,96.301370
Alley,1369,93.767123
Fence,1179,80.753425
FireplaceQu,690,47.260274


There are 5 columns of 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu' which are missing too many values (about 50%)

In [45]:
# drop 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'
house_data.drop(columns= ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'], inplace= True)

Seperate the dataset into 2 datasets of numerical and categorical features

In [48]:
# numerical dataset
num_df = house_data.select_dtypes(exclude= 'object')

# check if any missing value in the numeric dataset
num_df.isnull().sum().sort_values(ascending= False)

LotFrontage      259
GarageYrBlt       81
MasVnrArea         8
Id                 0
OpenPorchSF        0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageCars         0
GarageArea         0
WoodDeckSF         0
EnclosedPorch      0
HalfBath           0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
BedroomAbvGr       0
FullBath           0
MSSubClass         0
BsmtFinSF2         0
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
BsmtFinSF1         0
BsmtUnfSF          0
BsmtHalfBath       0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
SalePrice          0
dtype: int64

In [49]:
# categorical dataset
cat_df = house_data.select_dtypes(include= 'object')

cat_df['Id'] = house_data.Id

# check if any missing value
cat_df.isnull().sum().sort_values(ascending= False)

GarageCond       81
GarageQual       81
GarageFinish     81
GarageType       81
BsmtExposure     38
BsmtFinType2     38
BsmtCond         37
BsmtFinType1     37
BsmtQual         37
MasVnrType        8
Electrical        1
HeatingQC         0
Heating           0
MSZoning          0
CentralAir        0
Functional        0
PavedDrive        0
SaleType          0
SaleCondition     0
KitchenQual       0
Foundation        0
Street            0
ExterCond         0
LotShape          0
LandContour       0
Utilities         0
LotConfig         0
LandSlope         0
Neighborhood      0
Condition1        0
Condition2        0
BldgType          0
HouseStyle        0
RoofStyle         0
RoofMatl          0
Exterior1st       0
Exterior2nd       0
ExterQual         0
Id                0
dtype: int64

It looks like the missing values in 'LotFrontage' in the numerical dataset do not any connection to the categorical dataset; on the other hand, GarageYrBlt, MasVnrArea seem to have some connection with GarageCond, GarageQual, GarageFinish, GarageType, and MasVnrType. I will replace the missing values in LotFrontage with its mean.

In [50]:
# replace missing values in 'LotFrontage' with the mean of the remaining observations
num_df.LotFrontage.fillna(num_df.LotFrontage.mean(), inplace= True)

Since only 1 missing value in 'Electrical' feature, I will fill with the most frequent value.

In [75]:
cat_df['Electrical'].fillna(cat_df['Electrical'].mode()[0], inplace = True)

In [53]:
# check the missing values related to garage
pd.DataFrame(house_data[house_data['GarageCond'].isnull()].isnull().sum().sort_values(ascending= False)).head(5)

,0
GarageType,81
GarageYrBlt,81
GarageFinish,81
GarageQual,81
GarageCond,81


It looks like all of the missing values related to garage in the same houses as expected.

Based on the data description, the NA value in 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond' mean No Garage

In [76]:
garage_cat = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']

for g in garage_cat:
    cat_df[g] = cat_df[g].fillna('NG')

In [56]:
# Fill GarageYrBlt with -1
num_df.GarageYrBlt.fillna(-1, inplace= True)

In [77]:
pd.DataFrame(house_data[house_data.MasVnrType.isnull()].isnull().sum().sort_values(ascending= False)).head(2)

,0
MasVnrType,8
MasVnrArea,8


It looks like all of the missing values related to Masonry veneer in the same houses as expected. Based on the data description, the missing value in 'MasVnrType' mean No Masonry veneer

In [59]:
cat_df['MasVnrType'] = cat_df['MasVnrType'].fillna('NM')

In [60]:
num_df['MasVnrArea'] = num_df['MasVnrArea'].fillna(0)

According to the NaN check above, there are 38 missing values in each BsmtExposure and BsmtFinType2, and 37 missing values in each others. 

Now, I take a look at the missing values in BsmtQual.

In [61]:
pd.DataFrame(cat_df[cat_df['BsmtQual'].isnull()].isnull().sum().sort_values(ascending= False)).head(5)

,0
BsmtCond,37
BsmtExposure,37
BsmtFinType1,37
BsmtFinType2,37
BsmtQual,37


There are 37 no-basement houses. I will replace the NaN value with 'NB'.

In [62]:
# create a sub-dataset for that 37 houses
bsmt_missing_df = cat_df[cat_df['BsmtQual'].isnull()]

In [63]:
# remove bsmt_missing_df from cat_df
cat_df = pd.concat([cat_df, bsmt_missing_df, bsmt_missing_df]).drop_duplicates(keep = False)

In [64]:
# fill all missing value with 'NB', meaning No Basement
bsmt_missing_df = bsmt_missing_df.fillna('NB')

In [65]:
# join bsmt_missing_df
cat_df = cat_df.append(bsmt_missing_df)

C:\Users\alexb\AppData\Local\Temp\ipykernel_16668\534449588.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_df = cat_df.append(bsmt_missing_df)


Now, I check missing values in BsmtExposure and BsmtFintype2 that are not in the 37 houses above

In [67]:
cat_df[cat_df.BsmtExposure.isnull()].append(cat_df[cat_df.BsmtFinType2.isnull()]).T

C:\Users\alexb\AppData\Local\Temp\ipykernel_16668\196392462.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_df[cat_df.BsmtExposure.isnull()].append(cat_df[cat_df.BsmtFinType2.isnull()]).T


,948,332
MSZoning,RL,RL
Street,Pave,Pave
LotShape,IR1,IR1
LandContour,Lvl,Lvl
Utilities,AllPub,AllPub
LotConfig,Inside,Inside
LandSlope,Gtl,Gtl
Neighborhood,CollgCr,NridgHt
Condition1,Norm,Norm
Condition2,Norm,Norm


Since the other features relatived to basement existed, I will replace the missing values with the most common in the dataset

In [68]:
cat_df.BsmtExposure.fillna(cat_df.BsmtExposure.mode()[0], inplace= True)

In [69]:
cat_df.BsmtFinType2.fillna(cat_df.BsmtFinType2.mode()[0], inplace= True)

In [70]:
# re-check NaN
cat_df.isnull().sum().sort_values(ascending = False)

MSZoning         0
KitchenQual      0
BsmtExposure     0
BsmtFinType1     0
BsmtFinType2     0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
Functional       0
BsmtQual         0
GarageType       0
GarageFinish     0
GarageQual       0
GarageCond       0
PavedDrive       0
SaleType         0
SaleCondition    0
BsmtCond         0
Foundation       0
Street           0
Condition2       0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
BldgType         0
ExterCond        0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
ExterQual        0
Id               0
dtype: int64

In [78]:
# re-check NaN
num_df.isnull().sum().sort_values(ascending = False)

Id               0
WoodDeckSF       0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
OpenPorchSF      0
MSSubClass       0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
HalfBath         0
FullBath         0
BsmtHalfBath     0
BsmtFullBath     0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
SalePrice        0
dtype: int64

There is no NaN value in the dataset.

Now, I will join the numerical and categorical datasets into a new dataset called 'house_data_clean'.

In [71]:
# inner join the num_df and cat_df using 'Id', then drop 'Id' column
house_data_clean = num_df.join(cat_df.set_index('Id'), on = 'Id').drop(columns= 'Id')

In [72]:
house_data_clean.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,SBrkr,Gd,Typ,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
